In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import torch

In [2]:
where = Path.home() / "analysis" / "output"
judgement_group_id = "a7d83ead-4187-45ee-a81d-0ab63a8cd219"
interval ="10days"

def read_it(partition):
    return pd.read_parquet(where / f"judgement_group-{judgement_group_id}-{interval}-{partition}.parquet")

In [3]:
train,test = [read_it(partition) for partition in ('A','B')]

In [4]:
train

,id,text,label
0,feed_item/17e6dc5e-cba6-5c02-8251-db740c8a45dc,on the feasibility and robustness of pointwise...,0
1,feed_item/814603d2-0d8e-5394-a7da-30c302ff5d5a,to ice or not to ice icing can promote muscle ...,1
2,feed_item/d5e79aff-9b6d-518b-b7e2-e6aea1ebb037,a practical method to improve the accuracy of ...,1
3,feed_item/76157e6b-4974-55c4-9dd9-310dc2d369d9,a four legged robotic system for playing socce...,1
4,feed_item/0ad7d84b-8964-5703-9fdd-c4dc499fdd54,lipid nanoparticles stimulate innate immune ce...,0
...,...,...,...
2199,feed_item/3549dfcc-a1fc-5e8d-a013-bc86ff5b661c,defying the holocaust didn t just mean uprisin...,0
2200,feed_item/c381b7cd-520f-5484-aed9-3c1d26cbadb8,did we personalize assessing personalization b...,0
2201,feed_item/f50bc040-7e7a-535a-aeab-a7871c5057bb,mitochondria power supply failure may cause ag...,0
2202,feed_item/3334fb56-a491-5e49-9bd1-ebc60963962e,nuclear phase out would increase pollution dea...,1


In [5]:
test

,id,text,label,bow_proba,embed_proba
0,feed_item/16edc9cc-72f4-5011-9295-9a9d7835478a,tensor completion with provable consistency an...,1,0.333339,0.237094
1,feed_item/b82c6f39-7723-56ab-8d48-869642daf38a,benzema and asensio win first leg for real mad...,0,0.471106,0.713918
2,feed_item/eb81c20c-0db8-534c-8c6f-ba22ebb9109e,towards large scale simulations of open ended ...,0,0.484784,0.246378
3,feed_item/6c8b1b21-3368-5c5a-aa9d-3a79efa13f8c,littlebird efficient faster longer transformer...,1,0.589898,0.753596
4,feed_item/699b05cd-70fe-5e3c-9a85-80c1ecab12d8,an interrogation of identity as a hyphenated m...,0,0.457970,0.436548
...,...,...,...,...,...
295,feed_item/c89ceff7-6f2d-5ee5-86a3-96abeb978e3e,control invariant set enhanced reinforcement l...,0,0.265793,0.221919
296,feed_item/623e4953-5d49-5b6c-bf40-07f091394194,study unfair labor practices delay first union...,1,0.433425,0.302847
297,feed_item/485a8fe1-609f-5aa0-9e27-524fe8b5a4b5,he s ready ten hag backs martial to step up fo...,0,0.320109,0.624221
298,feed_item/6aba9761-2095-57de-b544-e1ec24b2e41e,europe is about to launch one of its most ambi...,1,0.412306,0.644858


In [6]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def to_tokenized_ds(that):
    ds = Dataset.from_dict({"text": train["text"], "label": train["label"]})
    dst = ds.map(tokenize_function)
    dst = dst.remove_columns("text")
    return dst

In [8]:
train_ds=to_tokenized_ds(train)

Map:   0%|          | 0/2204 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    num_train_epochs=3,
    logging_steps = 100,
    save_total_limit=5
)

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds
)

In [11]:
trainer.train()

C:\Users\paul\miniconda3\envs\yoshinon\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2204
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 828
  Number of trainable parameters = 108311810


Step,Training Loss
100,0.695400
200,0.677200
300,0.685500
400,0.677500
500,0.656900
600,0.624600
700,0.557200
800,0.533900


Saving model checkpoint to test_trainer\checkpoint-500
Configuration saved in test_trainer\checkpoint-500\config.json
Model weights saved in test_trainer\checkpoint-500\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=828, training_loss=0.634058610252712, metrics={'train_runtime': 136.2153, 'train_samples_per_second': 48.541, 'train_steps_per_second': 6.079, 'total_flos': 1739690298040320.0, 'train_loss': 0.634058610252712, 'epoch': 3.0})

In [12]:
from itertools import islice
 
 
def chunk(arr_range, arr_size):
    arr_range = iter(arr_range)
    return iter(lambda: tuple(islice(arr_range, arr_size)), ())

In [13]:
results=[]
with torch.no_grad():
    for chunk in chunk(list(test["text"]),100):
        inputs = tokenizer(chunk, return_tensors="pt", padding="max_length", truncation=True).to(0)
        result=model(**inputs).logits.cpu().detach().numpy()
        results += [result]

In [14]:
logits=np.concatenate(results,0)

In [15]:
def proba(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x[:,1] / e_x.sum(axis=1)

In [16]:
test["proba"]=proba(logits)

In [17]:
test.sort_values("proba",inplace=True)

In [18]:
pd.set_option('display.max_rows', None)

In [19]:
test

,id,text,label,bow_proba,embed_proba,proba
150,feed_item/4c970df3-0551-53a2-8294-6ba211d1dc07,optimal convergence analysis of laguerre spect...,0,0.338598,0.170401,0.022235
290,feed_item/50b4fa48-f635-564e-8f3e-61268d437190,zero truncated poisson regression for sparse m...,0,0.376413,0.248500,0.022740
211,feed_item/95f304cb-864d-5335-a3c6-23abc0644620,an adaptive factorized nystr om preconditioner...,0,0.287286,0.231489,0.022782
182,feed_item/08bbe51c-7d8c-5ac4-984d-74cea0b05a9c,state estimation of a carbon capture process t...,0,0.264628,0.217652,0.022859
90,feed_item/65527d04-1e4c-5cfc-bcaf-6e9f40b5f8de,strategy synthesis for zero sum neuro symbolic...,0,0.351846,0.185986,0.024081
253,feed_item/7866992e-74ed-52a9-86d2-a34cc2141b8e,a simple proof that the hp fem does not suffer...,0,0.253727,0.107857,0.025064
15,feed_item/28f1227e-7b14-5d90-a42d-fa5cef29e6c1,on parallel k center clustering we consider th...,0,0.254695,0.234549,0.027251
64,feed_item/8ceda510-ff77-560d-bc24-95923196c005,learning coordination through new actions we p...,0,0.279250,0.166590,0.029632
285,feed_item/43d26a4f-80e1-50f5-8005-a05b9ba59d84,vietnam sentences political activist to 6 year...,0,0.286909,0.311785,0.031411
146,feed_item/967585ba-bb7a-599a-bc85-ff9b339bf444,optimal testing of generalized reed muller cod...,0,0.388569,0.236149,0.035289


In [20]:
from sklearn.metrics import roc_auc_score

In [21]:
roc_auc_score(test["label"],test["proba"])

0.7227942194930514

In [22]:
roc_auc_score(test["label"],test["bow_proba"])

0.6631885128583961

In [23]:
roc_auc_score(test["label"],test["embed_proba"])

0.6824414792926728